In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 2)

os.chdir("/home/coinse/Workspace/SBFL")

# Load bug info

In [2]:
from scripts.util import load_bug_data, find_covmat_func, find_covmat_idx
from functools import reduce
from typing import Set

pid, vid = "proj", 5

bug = load_bug_data(pid, vid)

print(f"{bug.buggy_files=}")
print(f"{bug.buggy_lines=}")
print("bug.coverage_df=")
display(bug.coverage_df)
print(f"{bug.failing_tests=}")
print(f"{bug.failing_tests_GT=}")
print("bug.functions={", end="")
for func in list(bug.functions)[:5]:
    print(f"{func},", end=" ")
print(", ...}")
print(f"{bug.id=}")
print(f"{bug.num_test_cases=}")
print(f"{bug.passing_tests=}")
print(f"{bug.path_to_cov_df=}")
print(f"{bug.path_to_patch=}")
print(f"{bug.path_to_test_results=}")
print("bug.source_files=", end="")
for src in list(bug.source_files)[:5]:
    print(f"{src},", end=" ")
print(", ...}")

fault_idxs = sorted(bug.buggy_lines)
# fault_idx2meth = {}
fault_meth2idx = {}
err_msg = ""
for fault_idx in fault_idxs:
    succ_func, ret_func = find_covmat_func(
        pid, bug.coverage_df, fault_idx, debug=False
    )
    if succ_func:
        succ_idx, ret_idx = find_covmat_idx(
            pid, bug.coverage_df, fault_idx, debug=False
        )
        if succ_idx:
            if ret_func in fault_meth2idx:
                fault_meth2idx[ret_func].add(ret_idx)
            else:
                fault_meth2idx[ret_func] = {ret_idx}
        else:
            print(f"{ret_idx}")
            if ret_func not in fault_meth2idx:
                fault_meth2idx[ret_func] = set()
    else:
        print(f"{ret_func}")
# fault_meths: Set = reduce(
#     lambda x, y: x | set([y]), fault_idx2meth.values(), set()
# )
# fault_meth2idx = {
#     v: {idx for idx in fault_idx2meth if fault_idx2meth[idx] == v}
#     for v in fault_meths
# }
fault_meths = set(fault_meth2idx.keys())
for meth in fault_meth2idx:
    print(f"{meth=}: {fault_meth2idx[meth]}")
# print(f"{fault_meths=}")


bug.buggy_files={'src/iso19111/c_api.cpp'}
bug.buggy_lines={('src/iso19111/c_api.cpp', 7666), ('src/iso19111/c_api.cpp', 7667)}
bug.coverage_df=


2  \
file                                      function                             line      
/home/workspace/src/projections/times.cpp _ZL15times_s_forward5PJ_LPP8PJconsts 41    5   
                                                                               43    5   
                                                                               46    5   
                                                                               47    5   
                                                                               48    5   
...                                                                                 ..   
/home/workspace/src/projections/putp6.cpp pj_putp6                             91    2   
                                                                               93    2   
                                                                               94    2   
                                                                               95    2   
                                                                               97    2   

                                                                                     20  \
file                                      function                             line       
/home/workspace/src/projections/times.cpp _ZL15times_s_forward5PJ_LPP8PJconsts 41    0    
                                                                               43    0    
                                                                               46    0    
                                                                               47    0    
                                                                               48    0    
...                                                                                 ..    
/home/workspace/src/projections/putp6.cpp pj_putp6                             91    0    
                                                                               93    0    
                                                                               94    0    
                                                                               95    0    
                                                                               97    0    

                                                                                     30  \
file                                      function                             line       
/home/workspace/src/projections/times.cpp _ZL15times_s_forward5PJ_LPP8PJconsts 41    0    
                                                                               43    0    
                                                                               46    0    
                                                                               47    0    
                                                                               48    0    
...                                                                                 ..    
/home/workspace/src/projections/putp6.cpp pj_putp6                             91    0    
                                                                               93    0    
                                                                               94    0    
                                                                               95    0    
                                                                               97    0    

                                                                                     44  \
file                                      function                             line       
/home/workspace/src/projections/times.cpp _ZL15times_s_forward5PJ_LPP8PJconsts 41    0    
                                                                               43    0    
                                                                               46    0    
                                                                               47    0    


bug.failing_tests={'43'}
bug.failing_tests_GT={'43'}
bug.functions={('/home/workspace/src/iso19111/common.cpp', '_ZNK5osgeo4proj6common13UnitOfMeasureeqERKS2_'), ('/home/workspace/src/iso19111/factory.cpp', '_ZN5osgeo4proj2io15DatabaseContext7Private25getGeodeticDatumFromCacheERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'), ('/home/workspace/src/iso19111/crs.cpp', '_ZN5osgeo4proj3crs13ParametricCRS6createERKNS0_4util11PropertyMapERKN7dropbox6oxygen2nnISt10shared_ptrINS0_5datum15ParametricDatumEEEERKNS9_ISA_INS0_2cs12ParametricCSEEEE'), ('/home/workspace/src/filemanager.cpp', '_ZN5osgeo4proj14FileApiAdapter4readEPvm'), ('/home/workspace/src/iso19111/c_api.cpp', 'proj_create_conversion_tunisia_mapping_grid'), , ...}
bug.id='proj-5'
bug.num_test_cases=45
bug.passing_tests={'6', '42', '1', '15', '3', '45', '39', '35', '12', '7', '44', '27', '9', '5', '26', '2', '16', '8', '4', '19', '18', '33', '28', '22', '11', '14', '10', '24', '13', '30', '41', '32', '40', '37', '17', '34', '2

# Get spectra

In [3]:
covmat = bug.coverage_df
pass_tcs = bug.passing_tests
fail_tcs = bug.failing_tests

covmat_bool = covmat.astype(bool)
P = len(pass_tcs)
F = len(fail_tcs)
e_p = covmat_bool[pass_tcs].sum(axis=1)
e_f = covmat_bool[fail_tcs].sum(axis=1)
n_p = P - e_p
n_f = F - e_f
spectra = pd.DataFrame(
    {
        "line": covmat.index.to_list(),
        "e_p": e_p,
        "e_f": e_f,
        "n_p": n_p,
        "n_f": n_f,
    }
)

display(spectra)


line  \
file                                      function                             line                                                                                          
/home/workspace/src/projections/times.cpp _ZL15times_s_forward5PJ_LPP8PJconsts 41    (/home/workspace/src/projections/times.cpp, _ZL15times_s_forward5PJ_LPP8PJconsts, 41)   
                                                                               43    (/home/workspace/src/projections/times.cpp, _ZL15times_s_forward5PJ_LPP8PJconsts, 43)   
                                                                               46    (/home/workspace/src/projections/times.cpp, _ZL15times_s_forward5PJ_LPP8PJconsts, 46)   
                                                                               47    (/home/workspace/src/projections/times.cpp, _ZL15times_s_forward5PJ_LPP8PJconsts, 47)   
                                                                               48    (/home/workspace/src/projections/times.cpp, _ZL15times_s_forward5PJ_LPP8PJconsts, 48)   
...                                                                                                                                                                    ...   
/home/workspace/src/projections/putp6.cpp pj_putp6                             91    (/home/workspace/src/projections/putp6.cpp, pj_putp6, 91)                               
                                                                               93    (/home/workspace/src/projections/putp6.cpp, pj_putp6, 93)                               
                                                                               94    (/home/workspace/src/projections/putp6.cpp, pj_putp6, 94)                               
                                                                               95    (/home/workspace/src/projections/putp6.cpp, pj_putp6, 95)                               
                                                                               97    (/home/workspace/src/projections/putp6.cpp, pj_putp6, 97)                               

                                                                                     e_p  \
file                                      function                             line        
/home/workspace/src/projections/times.cpp _ZL15times_s_forward5PJ_LPP8PJconsts 41    1     
                                                                               43    1     
                                                                               46    1     
                                                                               47    1     
                                                                               48    1     
...                                                                                 ..     
/home/workspace/src/projections/putp6.cpp pj_putp6                             91    1     
                                                                               93    1     
                                                                               94    1     
                                                                               95    1     
                                                                               97    1     

                                                                                     e_f  \
file                                      function                             line        
/home/workspace/src/projections/times.cpp _ZL15times_s_forward5PJ_LPP8PJconsts 41    0     
                                                                               43    0     
                                                                               46    0     
                                                                               47    0     
                                                                               48    0     
...                                                           

# Compute suspiciousness scores

In [4]:
def ochiai(e_f, e_p, n_f, n_p):
    return e_f / np.sqrt(e_p + e_f) / np.sqrt(e_f + n_f) if e_f else 0.0


def op2(e_f, e_p, n_f, n_p):
    return e_f - e_p / (e_p + n_p + 1)

spectra["Sop2"] = spectra.apply(
    lambda row: op2(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Sochiai"] = spectra.apply(
    lambda row: ochiai(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Rop2"] = spectra.Sop2.rank(ascending=False, method="max")
spectra["Rochiai"] = spectra.Sochiai.rank(ascending=False, method="max")


# Line level SBFL

In [5]:
for fault_meth in fault_meth2idx:
    fault_idxs_of_meth = fault_meth2idx[fault_meth]
    print("fault meth:", fault_meth)
    for fault_idx in fault_idxs_of_meth:
        print("fault idx:", tuple(fault_idx))
    print("SBFL score of faulty method:")
    display(spectra.loc[fault_idxs_of_meth].reset_index(drop=True))

print("Top 5 SBFL scores:")
print("Op2:")
display(
    spectra.sort_values("Sop2", ascending=False).head(5).reset_index(drop=True)
)
print("Ochiai:")
display(
    spectra.sort_values("Sochiai", ascending=False)
    .head(5)
    .reset_index(drop=True)
)


fault meth: ('/home/workspace/src/iso19111/c_api.cpp', 'proj_get_suggested_operation')
fault idx: ('/home/workspace/src/iso19111/c_api.cpp', 'proj_get_suggested_operation', 7667)
fault idx: ('/home/workspace/src/iso19111/c_api.cpp', 'proj_get_suggested_operation', 7666)
SBFL score of faulty method:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation, 7667)",0,1,44,0,1.0,1.0,10549.0,10549.0
1,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation, 7666)",0,1,44,0,1.0,1.0,10549.0,10549.0


Top 5 SBFL scores:
Op2:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/iso19111/io.cpp, _ZN5osgeo4proj2io12WKTFormatterC2ENS2_10ConventionE, 318)",0,1,44,0,1.0,1.0,10549.0,10549.0
1,"(/home/workspace/src/iso19111/crs.cpp, _ZNK5osgeo4proj3crs12ProjectedCRS8identifyB5cxx11ERKSt10shared_ptrINS0_2io16AuthorityFactoryEE, 3620)",0,1,44,0,1.0,1.0,10549.0,10549.0
2,"(/home/workspace/src/iso19111/crs.cpp, _ZZNK5osgeo4proj3crs12ProjectedCRS8identifyB5cxx11ERKSt10shared_ptrINS0_2io16AuthorityFactoryEEENKUlPKcibE0_clESA_ib, 3649)",0,1,44,0,1.0,1.0,10549.0,10549.0
3,"(/home/workspace/src/iso19111/crs.cpp, _ZZNK5osgeo4proj3crs12ProjectedCRS8identifyB5cxx11ERKSt10shared_ptrINS0_2io16AuthorityFactoryEEENKUlPKcibE0_clESA_ib, 3648)",0,1,44,0,1.0,1.0,10549.0,10549.0
4,"(/home/workspace/src/iso19111/crs.cpp, _ZZNK5osgeo4proj3crs12ProjectedCRS8identifyB5cxx11ERKSt10shared_ptrINS0_2io16AuthorityFactoryEEENKUlPKcibE0_clESA_ib, 3647)",0,1,44,0,1.0,1.0,10549.0,10549.0


Ochiai:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/iso19111/io.cpp, _ZN5osgeo4proj2io9WKTParser7Private7buildCSERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureE, 2477)",0,1,44,0,1.0,1.0,10549.0,10549.0
1,"(/home/workspace/src/iso19111/crs.cpp, _ZN5osgeo4proj3crs19DerivedProjectedCRSC1ERKN7dropbox6oxygen2nnISt10shared_ptrINS1_12ProjectedCRSEEEERKNS5_IS6_INS0_9operation10ConversionEEEERKNS5_IS6_INS0_2cs16CoordinateSystemEEEE, 5181)",0,1,44,0,1.0,1.0,10549.0,10549.0
2,"(/home/workspace/src/iso19111/crs.cpp, _ZN5osgeo4proj3crs19DerivedProjectedCRS6createERKNS0_4util11PropertyMapERKN7dropbox6oxygen2nnISt10shared_ptrINS1_12ProjectedCRSEEEERKNS9_ISA_INS0_9operation10ConversionEEEERKNS9_ISA_INS0_2cs16CoordinateSystemEEEE, 5235)",0,1,44,0,1.0,1.0,10549.0,10549.0
3,"(/home/workspace/src/iso19111/crs.cpp, _ZN5osgeo4proj3crs19DerivedProjectedCRS6createERKNS0_4util11PropertyMapERKN7dropbox6oxygen2nnISt10shared_ptrINS1_12ProjectedCRSEEEERKNS9_ISA_INS0_9operation10ConversionEEEERKNS9_ISA_INS0_2cs16CoordinateSystemEEEE, 5234)",0,1,44,0,1.0,1.0,10549.0,10549.0
4,"(/home/workspace/src/iso19111/crs.cpp, _ZN5osgeo4proj3crs19DerivedProjectedCRS6createERKNS0_4util11PropertyMapERKN7dropbox6oxygen2nnISt10shared_ptrINS1_12ProjectedCRSEEEERKNS9_ISA_INS0_9operation10ConversionEEEERKNS9_ISA_INS0_2cs16CoordinateSystemEEEE, 5233)",0,1,44,0,1.0,1.0,10549.0,10549.0


# Method level SBFL

In [6]:
spectra["file"] = spectra.index.map(lambda x: x[0])
spectra["func"] = spectra.index.map(lambda x: x[1])
spectra["line"] = spectra.index.map(lambda x: x[2])
# (file, func) is meth
spectra["meth"] = spectra.apply(lambda row: (row.file, row.func), axis=1)
spectra.reset_index(drop=True, inplace=True)
display(spectra)

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth
0,41,1,0,43,1,-0.022222,0.0,34217.0,37928.0,/home/workspace/src/projections/times.cpp,_ZL15times_s_forward5PJ_LPP8PJconsts,"(/home/workspace/src/projections/times.cpp, _ZL15times_s_forward5PJ_LPP8PJconsts)"
1,43,1,0,43,1,-0.022222,0.0,34217.0,37928.0,/home/workspace/src/projections/times.cpp,_ZL15times_s_forward5PJ_LPP8PJconsts,"(/home/workspace/src/projections/times.cpp, _ZL15times_s_forward5PJ_LPP8PJconsts)"
2,46,1,0,43,1,-0.022222,0.0,34217.0,37928.0,/home/workspace/src/projections/times.cpp,_ZL15times_s_forward5PJ_LPP8PJconsts,"(/home/workspace/src/projections/times.cpp, _ZL15times_s_forward5PJ_LPP8PJconsts)"
3,47,1,0,43,1,-0.022222,0.0,34217.0,37928.0,/home/workspace/src/projections/times.cpp,_ZL15times_s_forward5PJ_LPP8PJconsts,"(/home/workspace/src/projections/times.cpp, _ZL15times_s_forward5PJ_LPP8PJconsts)"
4,48,1,0,43,1,-0.022222,0.0,34217.0,37928.0,/home/workspace/src/projections/times.cpp,_ZL15times_s_forward5PJ_LPP8PJconsts,"(/home/workspace/src/projections/times.cpp, _ZL15times_s_forward5PJ_LPP8PJconsts)"
...,...,...,...,...,...,...,...,...,...,...,...,...
37923,91,1,0,43,1,-0.022222,0.0,34217.0,37928.0,/home/workspace/src/projections/putp6.cpp,pj_putp6,"(/home/workspace/src/projections/putp6.cpp, pj_putp6)"
37924,93,1,0,43,1,-0.022222,0.0,34217.0,37928.0,/home/workspace/src/projections/putp6.cpp,pj_putp6,"(/home/workspace/src/projections/putp6.cpp, pj_putp6)"
37925,94,1,0,43,1,-0.022222,0.0,34217.0,37928.0,/home/workspace/src/projections/putp6.cpp,pj_putp6,"(/home/workspace/src/projections/putp6.cpp, pj_putp6)"
37926,95,1,0,43,1,-0.022222,0.0,34217.0,37928.0,/home/workspace/src/projections/putp6.cpp,pj_putp6,"(/home/workspace/src/projections/putp6.cpp, pj_putp6)"


## Max aggregation

In [7]:
print("[Op2]")
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_op2 = spectra_max_op2[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sop2"]
]
spectra_max_op2["Rop2"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_op2.loc[spectra_max_op2.meth.isin(fault_meths)].reset_index(
        drop=True
    )
)
print("Top 20 methods:")
display(spectra_max_op2.head(20).reset_index(drop=True))
print()
print("[Ochiai]")
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_ochiai = spectra_max_ochiai[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sochiai"]
]
spectra_max_ochiai["Rochiai"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_ochiai.loc[
        spectra_max_ochiai.meth.isin(fault_meths)
    ].reset_index(drop=True)
)
print("Top 20 methods:")
display(spectra_max_ochiai.head(20).reset_index(drop=True))


[Op2]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",7672,0,1,44,0,1.0,1128.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/src/wkt_parser.cpp, _Z12pj_wkt_errorP20pj_wkt_parse_contextPKc)",46,0,1,44,0,1.0,1128.0
1,"(/home/workspace/src/wkt2_parser.cpp, pj_wkt2_lex)",218,0,1,44,0,1.0,1128.0
2,"(/home/workspace/src/wkt2_parser.cpp, pj_wkt2_error)",54,0,1,44,0,1.0,1128.0
3,"(/home/workspace/src/wkt2_parser.cpp, _Z13pj_wkt2_parseRKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE)",66,0,1,44,0,1.0,1128.0
4,"(/home/workspace/src/wkt2_generated_parser.c, yytnamerr)",2549,0,1,44,0,1.0,1128.0
5,"(/home/workspace/src/wkt2_generated_parser.c, yysyntax_error)",2685,0,1,44,0,1.0,1128.0
6,"(/home/workspace/src/wkt2_generated_parser.c, yystrlen)",2480,0,1,44,0,1.0,1128.0
7,"(/home/workspace/src/wkt2_generated_parser.c, yystpcpy)",2500,0,1,44,0,1.0,1128.0
8,"(/home/workspace/src/wkt2_generated_parser.c, yydestruct)",2705,0,1,44,0,1.0,1128.0
9,"(/home/workspace/src/wkt2_generated_parser.c, pj_wkt2_parse)",3191,0,1,44,0,1.0,1128.0



[Ochiai]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",7672,0,1,44,0,1.0,1128.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/src/wkt_parser.cpp, _Z12pj_wkt_errorP20pj_wkt_parse_contextPKc)",46,0,1,44,0,1.0,1128.0
1,"(/home/workspace/src/wkt2_parser.cpp, pj_wkt2_lex)",218,0,1,44,0,1.0,1128.0
2,"(/home/workspace/src/wkt2_parser.cpp, pj_wkt2_error)",54,0,1,44,0,1.0,1128.0
3,"(/home/workspace/src/wkt2_parser.cpp, _Z13pj_wkt2_parseRKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE)",66,0,1,44,0,1.0,1128.0
4,"(/home/workspace/src/wkt2_generated_parser.c, yytnamerr)",2549,0,1,44,0,1.0,1128.0
5,"(/home/workspace/src/wkt2_generated_parser.c, yysyntax_error)",2685,0,1,44,0,1.0,1128.0
6,"(/home/workspace/src/wkt2_generated_parser.c, yystrlen)",2480,0,1,44,0,1.0,1128.0
7,"(/home/workspace/src/wkt2_generated_parser.c, yystpcpy)",2500,0,1,44,0,1.0,1128.0
8,"(/home/workspace/src/wkt2_generated_parser.c, yydestruct)",2705,0,1,44,0,1.0,1128.0
9,"(/home/workspace/src/wkt2_generated_parser.c, pj_wkt2_parse)",3191,0,1,44,0,1.0,1128.0


## Line2Method voting (weight: sbfl score / dense rank)

In [8]:
print("[Op2]")
ballot_op2 = {}
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2["Rop2D"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="dense"
)
spectra_max_op2["vote"] = spectra_max_op2.Sop2 / spectra_max_op2.Rop2D
for row in spectra_max_op2.itertuples():
    if row.meth not in ballot_op2:
        ballot_op2[row.meth] = row.vote
    else:
        ballot_op2[row.meth] += row.vote
L2M_op2 = pd.DataFrame(
    {"meth": list(ballot_op2.keys()), "vote": list(ballot_op2.values())}
)
L2M_op2["rank"] = L2M_op2.vote.rank(ascending=False, method="max")
L2M_op2.sort_values("rank", inplace=True)
L2M_op2.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(L2M_op2.loc[L2M_op2.meth == fault_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == fault_meth].reset_index(
            drop=True
        )
    )
print("Top 5 methods:")
display(L2M_op2.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_op2.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_op2.loc[L2M_op2.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == top_meth].reset_index(
            drop=True
        )
    )
print()
print("[Ochiai]")
ballot_ochiai = {}
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai["RochiaiD"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="dense"
)
spectra_max_ochiai["vote"] = (
    spectra_max_ochiai.Sochiai / spectra_max_ochiai.RochiaiD
)
for row in spectra_max_ochiai.itertuples():
    if row.meth not in ballot_ochiai:
        ballot_ochiai[row.meth] = row.vote
    else:
        ballot_ochiai[row.meth] += row.vote
L2M_ochiai = pd.DataFrame(
    {"meth": list(ballot_ochiai.keys()), "vote": list(ballot_ochiai.values())}
)
L2M_ochiai["rank"] = L2M_ochiai.vote.rank(ascending=False, method="max")
L2M_ochiai.sort_values("rank", inplace=True)
L2M_ochiai.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(
        L2M_ochiai.loc[L2M_ochiai.meth == fault_meth].reset_index(drop=True)
    )
    print("Voters:")
    display(
        spectra_max_ochiai.loc[
            spectra_max_ochiai.meth == fault_meth
        ].reset_index(drop=True)
    )
print("Top 5 methods:")
display(L2M_ochiai.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_ochiai.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_ochiai.loc[L2M_ochiai.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_ochiai.loc[spectra_max_ochiai.meth == top_meth].reset_index(
            drop=True
        )
    )


[Op2]
Faulty method:


,meth,vote,rank
0,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",10.0,329.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,Rop2D,vote
0,7672,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/c_api.cpp,proj_get_suggested_operation,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",1.0,1.0
1,7670,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/c_api.cpp,proj_get_suggested_operation,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",1.0,1.0
2,7669,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/c_api.cpp,proj_get_suggested_operation,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",1.0,1.0
3,7667,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/c_api.cpp,proj_get_suggested_operation,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",1.0,1.0
4,7666,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/c_api.cpp,proj_get_suggested_operation,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",1.0,1.0
5,7665,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/c_api.cpp,proj_get_suggested_operation,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",1.0,1.0
6,7659,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/c_api.cpp,proj_get_suggested_operation,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",1.0,1.0
7,7658,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/c_api.cpp,proj_get_suggested_operation,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",1.0,1.0
8,7657,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/c_api.cpp,proj_get_suggested_operation,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",1.0,1.0
9,7655,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/c_api.cpp,proj_get_suggested_operation,"(/home/workspace/src/iso19111/c_api.cpp, proj_get_suggested_operation)",1.0,1.0


Top 5 methods:


...

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,3655,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/io.cpp,_ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_,"(/home/workspace/src/iso19111/io.cpp, _ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_)",1.0,1.0
1,3653,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/io.cpp,_ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_,"(/home/workspace/src/iso19111/io.cpp, _ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_)",1.0,1.0
2,3652,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/io.cpp,_ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_,"(/home/workspace/src/iso19111/io.cpp, _ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_)",1.0,1.0
3,3647,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/io.cpp,_ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_,"(/home/workspace/src/iso19111/io.cpp, _ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_)",1.0,1.0
4,3646,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/io.cpp,_ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_,"(/home/workspace/src/iso19111/io.cpp, _ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_)",1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,3407,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/io.cpp,_ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_,"(/home/workspace/src/iso19111/io.cpp, _ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_)",1.0,1.0
143,3406,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/io.cpp,_ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_,"(/home/workspace/src/iso19111/io.cpp, _ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_)",1.0,1.0
144,3405,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/io.cpp,_ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_,"(/home/workspace/src/iso19111/io.cpp, _ZN5osgeo4proj2io9WKTParser7Private23buildProjectionStandardERKN7dropbox6oxygen2nnISt10unique_ptrINS1_7WKTNodeESt14default_deleteIS8_EEEESE_RKNS0_6common13UnitOfMeasureESI_)",1.0,1.0
145,3403,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/io.cpp,_ZN5osgeo4p

Top 5:


,meth,vote,rank
0,"(/home/workspace/src/iso19111/coordinateoperation.cpp, _ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi)",141.0,5.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,5416,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/coordinateoperation.cpp,_ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi,"(/home/workspace/src/iso19111/coordinateoperation.cpp, _ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi)",1.0,1.0
1,5415,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/coordinateoperation.cpp,_ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi,"(/home/workspace/src/iso19111/coordinateoperation.cpp, _ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi)",1.0,1.0
2,5414,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/coordinateoperation.cpp,_ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi,"(/home/workspace/src/iso19111/coordinateoperation.cpp, _ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi)",1.0,1.0
3,5412,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/coordinateoperation.cpp,_ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi,"(/home/workspace/src/iso19111/coordinateoperation.cpp, _ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi)",1.0,1.0
4,5411,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/coordinateoperation.cpp,_ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi,"(/home/workspace/src/iso19111/coordinateoperation.cpp, _ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi)",1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,5191,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/coordinateoperation.cpp,_ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi,"(/home/workspace/src/iso19111/coordinateoperation.cpp, _ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi)",1.0,1.0
137,5188,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/coordinateoperation.cpp,_ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi,"(/home/workspace/src/iso19111/coordinateoperation.cpp, _ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi)",1.0,1.0
138,5187,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/coordinateoperation.cpp,_ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi,"(/home/workspace/src/iso19111/coordinateoperation.cpp, _ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi)",1.0,1.0
139,5186,0,1,44,0,1.0,1.0,10549.0,10549.0,/home/workspace/src/iso19111/coordinateoperation.cpp,_ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi,"(/home/workspace/src/iso19111/coordinateoperation.cpp, _ZNK5osgeo4proj9operation10Conversion20convertToOtherMethodEi)",1.0,1.0


# 감상

Max aggregation으로 top이지만 그게 1128개. line 수가 많은 편이 아니어서 voting으로도 성적이 좋지 않음